<a href="https://colab.research.google.com/github/cs1090218/conv/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load the model (might take 5-10 mins for non-tiny model)

In [8]:
# If the local dir doesn't have adapter files and we need to get the full model from hub

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "shashankverma590/qwen-0_5b-kid-friendly-chatbot"             # 0.2 - 0.6 second for each response
# model_name = "shashankverma590/llama-3-1-8b-kid-friendly-chatbot"        # 6-10 seconds for each response
# model_name = "shashankverma590/tiny-llama-1b-kid-friendly-chatbot-tiny"    # 2-6 seconds for each response

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/4.64k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/509 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

# ChatBot

In [9]:
from transformers import pipeline
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime

system_prompt = """You are a helpful chatbot for conversing with kids under the age of 7.
You should be empathetic, encouraging and positive minded in general.
The current mood of the user is "{emotion}", you should reply accordingly."""

emotion = input('Enter the current mood: ')
conversation_history = [
    {"role": "system", "content": system_prompt.format(emotion=emotion)}
]

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
def get_chatbot_response(user_input):
    conversation_history.append({"role": "user", "content": user_input})
    prompt = pipe.tokenizer.apply_chat_template(conversation_history, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=False, temperature=0.1, top_k=50, top_p=0.1, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    response = outputs[0]['generated_text'][len(prompt):].strip()
    conversation_history.append({"role": "assistant", "content": response})
    return response


# Create widgets for user input and chatbot interaction
input_box = widgets.Text(
    description="You:",
    placeholder="Type your message here..."
)

output_box = widgets.Output()

def on_submit(change):
    with output_box:
        prompt = input_box.value
        input_box.value = ""
        print(f"You: {prompt}")

        # Get chatbot's response and print it
        start_time = datetime.now()
        response = get_chatbot_response(prompt)
        end_time = datetime.now()

        print(f"Chatbot: {response}\t\t\t({(end_time-start_time).total_seconds():.4f}s)")

# Set the function to trigger on user input
input_box.on_submit(on_submit)

# Display the widgets
display(input_box, output_box)


Enter the current mood: happy


Text(value='', description='You:', placeholder='Type your message here...')

Output()